### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [2]:
pd.options.display.max_columns= None
pd.options.display.max_colwidth= None
pd.options.display.max_rows = None

## Read dataset

In [3]:
df = pd.read_excel('/home/mahdi/NLP/Text_Preprocessing/dataset/Comment_Full_Data_Sentiment_Labeled.xlsx')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1131 entries, 0 to 1130
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   comment_id              1131 non-null   int64  
 1   app_id                  1131 non-null   int64  
 2   user_name               1131 non-null   object 
 3   comment_text            1131 non-null   object 
 4   Annotator1              1130 non-null   float64
 5   Annotator2              1129 non-null   float64
 6   Annotator3              1131 non-null   int64  
 7   comment_rating          1131 non-null   int64  
 8   comment_date            1131 non-null   int64  
 9   sentiment_result        1131 non-null   object 
 10  second_model_processed  1131 non-null   bool   
 11  comment_idd             1131 non-null   int64  
 12  sentiment_score         1131 non-null   int64  
 13  comment_date_jalali     1131 non-null   int64  
dtypes: bool(1), float64(2), int64(8), object

In [5]:
df.head()

,comment_id,app_id,user_name,comment_text,Annotator1,Annotator2,Annotator3,comment_rating,comment_date,sentiment_result,second_model_processed,comment_idd,sentiment_score,comment_date_jalali
0,111177,9,هادی,سلام،خوبه ولی برنامه همراه شهر باز نمیشه.,3.0,3.0,3,5,45650,mixed,False,175629681,0,14031004
1,111172,9,Unnamed User,سلام خسته نباشید از این برنامه میشه غیر حضوری حساب باز کنیم یا نه ؟,0.0,0.0,0,5,45649,no sentiment expressed,False,175659112,0,14031003
2,110746,8,Unnamed User,لطفا تعداد اقساط رو از ۱۲ ماه شروع کنید نه ۲۴ ماه,0.0,0.0,0,5,45651,no sentiment expressed,False,175712781,0,14031005
3,110706,8,shayan,خود بانک و وام هاش خوبه ولی نام افزارش خیلی هنگی و قطعی داره,3.0,3.0,3,2,45651,mixed,False,175740844,0,14031005
4,110702,8,hamed,سلام چرا وارد همراه بانک نمیتونم بشم,2.0,2.0,2,3,45651,no sentiment expressed,False,175741764,0,14031005


## Preprocessing

In [6]:
def _multiple_replace(mapping, text):
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

def convert_fa_numbers(input_str):
    mapping = {
        '۰': '0',
        '۱': '1',
        '۲': '2',
        '۳': '3',
        '۴': '4',
        '۵': '5',
        '۶': '6',
        '۷': '7',
        '۸': '8',
        '۹': '9',
        '.': '.',
    }
    return _multiple_replace(mapping, input_str)

def convert_en_numbers(input_str):
    mapping = {
         '0': '۰',
         '1' : '۱',
         '2' :'۲',
        '3'  :'۳',
        '4'  :'۴',
        '5' :'۵',
        '6' :'۶',
        '7' :'۷',
        '8' :'۸',
        '9' :'۹',
        '.' :'.'
    }
    return _multiple_replace(mapping, input_str)

def convert_ar_characters(input_str):
    """
    Converts Arabic chars to related Persian unicode char
    """
    mapping = {
        'ك': 'ک',
        'ى': 'ی',
        'ي': 'ی',
        'ئ':'ی',
        'إ':'ا',
        'أ':'ا',
        'ة':'ه',
        'ؤ':'و'
    }
    return _multiple_replace(mapping, input_str)



### Convert Emojis

In [21]:
import emoji
import re

def convert_emojis_to_persian(text):
    # First, convert emojis to English descriptive labels.
    # This will convert "😊" into something like ":smiling_face_with_smiling_eyes:".
    demojized_text = emoji.demojize(text)
    
    # Define a dictionary mapping some common English emoji labels to Persian words.
    persian_emoji_map = {
        ":smiling_face_with_smiling_eyes:": "خندان",
        ":grinning_face:": "با لبخند",
        ":face_with_tears_of_joy:": "با اشک شوق",
        ":red_heart:": "عشق",
        ":thumbs_up:": "پسندیده",
        # Add additional mappings as needed.
    }
    
    # Optionally, remove colons to clean up the text (if desired)
    for eng_label, pers_label in persian_emoji_map.items():
        pers_label = '['+pers_label+']'
        # Use regex to ensure exact matching of the label
        demojized_text = re.sub(re.escape(eng_label), pers_label, demojized_text)
    
    return demojized_text


## preprocess functions

In [83]:

def merge_mi_prefix(text):

    return re.sub(r'\b(ن?می)\s+(\S+)', r'\1\2', text)

def remove_diacritics(text):
    # Define regex for Persian diacritics (Unicode range: \u064B-\u0652)
    diacritics_pattern = re.compile(r'[\u064B-\u0652]')
    return re.sub(diacritics_pattern, '', text)


def convert_number_to_text(text):
    
    return re.sub(r'(\d)\d*', r'\1', text)



def map_num_to_text(text):

    # Check if the text is exactly '1', '2', or '3' (nothing else)
    mapping = {'1': 'خیلی بد', '2': 'بد', '3': 'متوسط' , '4': 'خوب' , '5': 'عالی'}
    
    if text in mapping:
        return mapping[text]  # Replace number with label
    
    return text  # Return original text if it’s not a single number


In [72]:
text = 'سلام. این شماره ۵ مال کدوم شماره از ۱۲۳۴۵ هست'

In [73]:
convert_number_to_text(text)

'سلام. این شماره ۵ مال کدوم شماره از ۱ هست'

In [84]:
# Example inputs:
print(map_num_to_text("1"))        # Output: خیلی بد
# print(map_number_to_text(" 2 "))      # Output: بد
# print(map_number_to_text("3"))        # Output: متوسط
# print(map_number_to_text("سلام 1"))   # Output: سلام 1
# print(map_number_to_text("۱۲۳"))      # Output: ۱۲۳
# print(map_number_to_text(" شماره 3 ")) # Output:  شماره 3 

خیلی بد


In [81]:
# text = df[df["comment_id"]== 93575]['comment_text']
text = df[df["comment_id"]== 94729]['comment_text']
text

1009    در هنگام ثبت نام کد دعوت yg6iamnz بزنید تا ۷۰ هزارتومان پاداش براتون واریز شه با تشکر ✅️✅️yg6iamnz✅️✅️
Name: comment_text, dtype: object

In [85]:
text.apply(lambda x : map_num_to_text(x))

1009    در هنگام ثبت نام کد دعوت yg6iamnz بزنید تا ۷۰ هزارتومان پاداش براتون واریز شه با تشکر ✅️✅️yg6iamnz✅️✅️
Name: comment_text, dtype: object

## Main Function

In [86]:
def preprocess(text,
               convert_farsi_numbers = False,
               convert_english_numbers = False,
               convert_arabic_characters = False,
               remove_diacritic = False,
               convert_emojis = False,
               remove_half_space = False,
               remove_removelist = False,
               remove_extra_characters = False,
               remove_numbers = False,
               remove_punctuation = False,
               replace_multiple_space = False,
               handle_prefix = False,
               map_number_to_text = False
               
               ):
    
    text = text.strip()

    if convert_farsi_numbers:
        text = convert_fa_numbers(text)

    if convert_arabic_characters:
# convert arabic characters to persian
        text = convert_ar_characters(text)

    if remove_diacritic:
        text = remove_diacritics(text)

    if convert_emojis:
# convert Emojis
        text = convert_emojis_to_persian(text)
    
    if remove_removelist:
        removelist = "<>"
        # text = re.sub(r'[^\w'+removelist+']', ' ', text)
        # text = re.sub(r'[^\w]', ' ', text)
        # text = re.sub(r'((#)[\w]*)','#',text)
    
    if remove_half_space:
# remove half space
        text = text.replace('\u200c', '')
    
    if remove_extra_characters:
        text = re.sub(r'(\w)\1{2,}', r'\1\1',text)

    if map_number_to_text:
        text = map_num_to_text(text)

# remove numbers
    if remove_numbers:
        text = re.sub(r' [\d+]', ' ',text)

# convert english numbers to persian
    if convert_english_numbers:
        text = convert_en_numbers(text)
    
# remove punctuations
    if remove_punctuation:
        text= re.sub(r'[^\w\[\]]', ' ', text)

# replace multiple spaces with one space
    if replace_multiple_space:
        text = re.sub(r'[\s]{2,}', ' ', text)

# prefix
    if handle_prefix:
        text = merge_mi_prefix(text)
   
    
    return(text)

## Test the function

In [46]:
text = ' سلامممم.!!! من نمی خواهم ازین جا ببببرم.... نمره ۵ می دم!؟.'

In [47]:
preprocess(text)

'سلامم من نمیخواهم ازین جا ببرم نمره 5 میدم '

In [48]:
# Example Usage
text_with_diacritics = "مُسلم، مَدرسه، اَمسْ"
clean_text = remove_diacritics(text_with_diacritics)
print(clean_text)  

مسلم، مدرسه، امس


In [49]:
text = df[df["comment_id"]== 110441]['comment_text']
# text = '👍👍👍👍'
text

21    برنامه خوب و بروز👍👍👍👍
Name: comment_text, dtype: object

In [50]:
# convert_emojis_to_persian(text)

In [51]:
preprocess(text)

'21 برنامه خوب و بروز[پسندیده][پسندیده][پسندیده][پسندیده] Name comment_text dtype object'